In [8]:
%pip install --upgrade google-cloud-firestore
%pip install pandas
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
from google.cloud import firestore
db = firestore.Client(project='events-explorer-448816', database="events-db-de")

In [10]:
import pandas as pd
df = pd.read_csv("germany_locations.csv")
df.head()

,priority,collection_id
0,2,Aach Baden-Württemberg
1,2,Aach Rheinland-Pfalz
2,1,Aachen Nordrhein-Westfalen
3,2,Aalen Baden-Württemberg
4,2,Aarbergen Hessen


In [11]:


# Function to map priority values to string labels
def map_priority(priority_value):
  """Maps numerical priority to string representation.

  Args:
      priority_value: The numerical priority value (e.g., 1.0, 2.0).

  Returns:
      A string representing the priority level ('High', 'Medium', or 'Low').
  """
  if priority_value == 1.0:
    return 'high'
  elif priority_value == 2.0:
    return 'medium'
  else:
      return 'low'

# Apply the mapping function to the 'priority' column
df['priority'] = df['priority'].apply(map_priority)

In [12]:
samples = df[0:2]
samples

,priority,collection_id
0,medium,Aach Baden-Württemberg
1,medium,Aach Rheinland-Pfalz


In [14]:
import datetime
for index, location in df.iterrows():
    
    coll_id = location["collection_id"]
    coll_id = coll_id.replace("/", " ")
    print(index, coll_id)
    doc_ref = db.collection("locations").document(coll_id)
    doc_ref.set({"num_events": 0, "priority": location["priority"], u"last_scanned": datetime.datetime.fromtimestamp(0, tz=datetime.timezone.utc)})

    doc_ref_stats = db.collection("locations").document("0_stats")
    doc_ref_stats.update({
        "num_locations": firestore.Increment(1)
    })

0 Aach Baden-Württemberg
1 Aach Rheinland-Pfalz
2 Aachen Nordrhein-Westfalen
3 Aalen Baden-Württemberg
4 Aarbergen Hessen
5 Aasbüttel Schleswig-Holstein
6 Abenberg Bayern
7 Abensberg Bayern
8 Abentheuer Rheinland-Pfalz
9 Absberg Bayern
10 Abstatt Baden-Württemberg
11 Abtsbessingen Thüringen
12 Abtsgmünd Baden-Württemberg
13 Abtsteinach Hessen
14 Abtswind Bayern
15 Abtweiler Rheinland-Pfalz
16 Achberg Baden-Württemberg
17 Achern Baden-Württemberg
18 Achim Niedersachsen
19 Achslach Bayern
20 Achstetten Baden-Württemberg
21 Acht Rheinland-Pfalz
22 Achtelsbach Rheinland-Pfalz
23 Achterwehr Schleswig-Holstein
24 Achtrup Schleswig-Holstein
25 Adelberg Baden-Württemberg
26 Adelebsen Niedersachsen
27 Adelheidsdorf Niedersachsen
28 Adelmannsfelden Baden-Württemberg
29 Adelschlag Bayern
30 Adelsdorf Bayern
31 Adelsheim Baden-Württemberg
32 Adelshofen Bayern
33 Adelsried Bayern
34 Adelzhausen Bayern
35 Adenau Rheinland-Pfalz
36 Adenbach Rheinland-Pfalz
37 Adenbüttel Niedersachsen
38 Adendorf Nied

In [15]:
event_types = [
    {"type": "Festival", "description": "Large-scale music or cultural festival"},
    {"type": "Exhibition or Trade Fair", "description": "Large gathering of people for a specific industry or interest"},
    {"type": "Conference", "description": "Business or academic conference with presentations and networking opportunities"},
    {"type": "Public Holiday Celebration", "description": "Events and gatherings related to national or religious holidays"},
    {"type": "Market", "description": "Regularly occurring market with vendors selling goods, including farmers' markets, Christmas markets, etc."},
    {"type": "Football Match", "description": "Professional or amateur football match in a stadium"},
    {"type": "Motorsports Race", "description": "Car or motorcycle race event"},
    {"type": "Marathon or Running Event", "description": "Organized race with a large number of participants"},
    {"type": "Music Festival", "description": "Outdoor music festival with multiple stages and camping"},
    {"type": "Concert", "description": "Indoor or outdoor concert featuring a single artist or band"},
    {"type": "Classical Music Performance", "description": "Performance of orchestral music, opera, or chamber music"},
    {"type": "Theater Performance", "description": "Play or musical performed in a theater"},
    {"type": "Museum Exhibition", "description": "Special exhibition or event at a museum"},
    {"type": "Art Festival", "description": "Festival showcasing visual arts, installations, and performances"},
    {"type": "Political Rally", "description": "Gathering of people for a political party or candidate"},
    {"type": "Religious Gathering", "description": "Religious service or celebration with a large number of attendees"},
    {"type": "Wedding", "description": "Large wedding celebration with many guests"},
    {"type": "Parade", "description": "Organized procession of people and vehicles for a celebration or protest"},
    {"type": "Oktoberfest", "description": "World-famous beer festival in Munich"},
    {"type": "Karneval or Fasching", "description": "Carnival celebrations in various regions of Germany"},
    {"type": "Weihnachtsmarkt", "description": "Traditional Christmas markets held in many German cities"},
    {"type": "Kirmes", "description": "Local fair with amusement rides and food stalls"},
    {"type": "Volksfest", "description": "Traditional folk festival with rides, food, and entertainment"}
]
for event_type in event_types:
    doc_ref = db.collection("event_types").document(event_type["type"])
    doc_ref.set(event_type)

In [2]:
import json
event = json.loads("""{
        "address": "Mannheim, Germany",
        "date": "2025-06-07 to 2025-06-09",
        "end_time": "N/A",
        "event_type": "Art, Technology, Culture Festival",
        "name": "APEX Mannheim",
        "size": "L",
        "start_time":"N/A",
        "url": "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtPBU-Bc7Xmbbk1-gxUDYlvH99caaXeaw-40fT1QbtIb17JYGO_7j32zsVTf9yglZl8ySAAfGkqCuWMa9R8bw_dGalv3OWtqcBnQDDh9Hcl1qExdcleBPyjtJUjJl2XvKe1AG9mBPbtE--ECGJsgZuuwwLZBHisKma376MAOJXzXRlVn8hX"
    }""")

In [6]:
import pandas as pd
df = pd.DataFrame.from_dict(event, orient = "index").T
df

,address,date,end_time,event_type,name,size,start_time,url
0,"Mannheim, Germany",2025-06-07 to 2025-06-09,N/A,"Art, Technology, Culture Festival",APEX Mannheim,L,N/A,https://vertexaisearch.cloud.google.com/ground...
